# 📦 Losgrößenoptimierung in der Produktionsplanung

**Intelligente Produktionsplanung und KI**  
Universität Stuttgart  
Master Maschinenbau / Technologiemanagement

---

## Lernziele

Nach Bearbeitung dieses Notebooks können Sie:

1. Das **EOQ-Modell** (Economic Order Quantity) erklären und anwenden
2. Die **Kostentrade-offs** zwischen Bestell- und Lagerkosten verstehen
3. Die **optimale Losgröße** und zugehörige Kennzahlen berechnen
4. Eine **Sensitivitätsanalyse** durchführen und interpretieren
5. Das EOQ-Modell um **Mengenrabatte** und **Sicherheitsbestände** erweitern

---

## 1. Einführung und Grundlagen

### Was ist Losgrößenoptimierung?

Die **Losgrößenoptimierung** befasst sich mit der Frage: **Wie viel soll auf einmal bestellt/produziert werden?**

### Das Dilemma der Losgröße

| Große Lose | Kleine Lose |
|------------|-------------|
| ✅ Weniger Bestellungen | ✅ Weniger Kapitalbindung |
| ✅ Geringere Bestellkosten | ✅ Geringere Lagerkosten |
| ❌ Hohe Lagerbestände | ❌ Häufige Bestellungen |
| ❌ Hohe Kapitalbindung | ❌ Hohe Bestellkosten |

### Das EOQ-Modell (Economic Order Quantity)

Das EOQ-Modell, auch **Andler-Formel** oder **Harris-Wilson-Formel** genannt, findet die **kostenoptimale Bestellmenge**, die den Trade-off zwischen Bestell- und Lagerkosten minimiert.

### Annahmen des klassischen EOQ-Modells

1. **Konstanter Bedarf**: Die Nachfrage ist bekannt und gleichmäßig
2. **Keine Fehlmengen**: Lieferung erfolgt rechtzeitig
3. **Sofortige Lieferung**: Keine Lieferzeit (oder bekannte, konstante Lieferzeit)
4. **Konstante Kosten**: Bestell- und Lagerkosten ändern sich nicht
5. **Keine Mengenrabatte**: Der Stückpreis ist unabhängig von der Bestellmenge

In [ ]:
# ===================================================================
# Imports und Konfiguration
# ===================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

# Matplotlib Konfiguration
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 11
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Alle Bibliotheken erfolgreich geladen!")
print("\n📦 Verwendete Pakete:")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")

---

## 2. Mathematische Herleitung des EOQ-Modells

### Notation

| Symbol | Bedeutung | Einheit |
|--------|-----------|--------|
| $D$ | Jahresbedarf (Demand) | Stück/Jahr |
| $K$ | Bestellkosten pro Bestellung (Order Cost) | €/Bestellung |
| $h$ | Lagerhaltungskosten pro Stück und Jahr (Holding Cost) | €/Stück/Jahr |
| $Q$ | Losgröße (Order Quantity) | Stück |
| $Q^*$ | Optimale Losgröße | Stück |

### Kostenkomponenten

**1. Bestellkosten pro Jahr:**
$$K_{Bestell} = \frac{D}{Q} \cdot K$$

- $\frac{D}{Q}$ = Anzahl der Bestellungen pro Jahr
- Sinken mit steigender Losgröße

**2. Lagerkosten pro Jahr:**
$$K_{Lager} = \frac{Q}{2} \cdot h$$

- $\frac{Q}{2}$ = Durchschnittlicher Lagerbestand (bei gleichmäßigem Verbrauch)
- Steigen mit steigender Losgröße

**3. Gesamtkosten pro Jahr:**
$$K_{Gesamt}(Q) = \frac{D}{Q} \cdot K + \frac{Q}{2} \cdot h$$

### Herleitung der optimalen Losgröße

Minimum durch Ableitung und Nullsetzen:

$$\frac{dK_{Gesamt}}{dQ} = -\frac{D \cdot K}{Q^2} + \frac{h}{2} = 0$$

Auflösen nach $Q$:

$$\boxed{Q^* = \sqrt{\frac{2 \cdot D \cdot K}{h}}}$$

### Minimale Gesamtkosten

Einsetzen von $Q^*$ in die Kostenfunktion:

$$\boxed{K^*_{Gesamt} = \sqrt{2 \cdot D \cdot K \cdot h}}$$

**Bemerkenswert:** Bei der optimalen Losgröße sind Bestellkosten = Lagerkosten!

In [ ]:
# ===================================================================
# EOQ-Funktionen implementieren
# ===================================================================

def eoq_kosten(Q, D, K, h):
    """
    Berechnet die Gesamtkosten für eine gegebene Losgröße Q.
    
    Parameter:
    ----------
    Q : float
        Losgröße (Stück pro Bestellung)
    D : float
        Jahresbedarf (Stück/Jahr)
    K : float
        Bestellkosten pro Bestellung (€)
    h : float
        Lagerhaltungskosten pro Stück und Jahr (€/Stück/Jahr)
        
    Returns:
    --------
    dict : Dictionary mit allen Kostenkomponenten
    """
    if Q <= 0:
        return {'Bestellkosten': float('inf'), 
                'Lagerkosten': float('inf'), 
                'Gesamtkosten': float('inf')}
    
    bestellkosten = (D / Q) * K
    lagerkosten = (Q / 2) * h
    gesamtkosten = bestellkosten + lagerkosten
    
    return {
        'Bestellkosten': bestellkosten,
        'Lagerkosten': lagerkosten,
        'Gesamtkosten': gesamtkosten
    }

def eoq_optimal(D, K, h):
    """
    Berechnet die optimale Losgröße und zugehörige Kennzahlen.
    
    Parameter:
    ----------
    D : float
        Jahresbedarf (Stück/Jahr)
    K : float
        Bestellkosten pro Bestellung (€)
    h : float
        Lagerhaltungskosten pro Stück und Jahr (€/Stück/Jahr)
        
    Returns:
    --------
    dict : Dictionary mit allen Ergebnissen
    """
    # Optimale Losgröße (EOQ-Formel)
    Q_opt = np.sqrt((2 * D * K) / h)
    
    # Kosten bei optimaler Losgröße
    kosten = eoq_kosten(Q_opt, D, K, h)
    
    # Weitere Kennzahlen
    bestellhaeufigkeit = D / Q_opt  # Bestellungen pro Jahr
    bestellzyklus = 365 / bestellhaeufigkeit  # Tage zwischen Bestellungen
    durchschnittlicher_bestand = Q_opt / 2
    
    return {
        'Q_optimal': Q_opt,
        'Bestellkosten': kosten['Bestellkosten'],
        'Lagerkosten': kosten['Lagerkosten'],
        'Gesamtkosten': kosten['Gesamtkosten'],
        'Bestellhaeufigkeit': bestellhaeufigkeit,
        'Bestellzyklus_Tage': bestellzyklus,
        'Durchschn_Bestand': durchschnittlicher_bestand
    }

print("✅ EOQ-Funktionen definiert:")
print("   • eoq_kosten(Q, D, K, h) - Berechnet Kosten für gegebene Losgröße")
print("   • eoq_optimal(D, K, h)   - Berechnet optimale Losgröße und Kennzahlen")

---

## 3. Fallbeispiel: Automobilzulieferer GmbH

### Problemstellung

Die **Automobilzulieferer GmbH** produziert verschiedene Komponenten für die Automobilindustrie. Für drei Hauptprodukte soll die optimale Bestellpolitik ermittelt werden:

| Produkt | Jahresbedarf (D) | Bestellkosten (K) | Lagerkosten (h) |
|---------|------------------|-------------------|------------------|
| Bremssattel | 12.000 Stück | 150 € | 8,50 €/Stück/Jahr |
| Stoßdämpfer | 8.000 Stück | 200 € | 12,00 €/Stück/Jahr |
| Kupplungsscheibe | 15.000 Stück | 120 € | 6,80 €/Stück/Jahr |

In [ ]:
# ===================================================================
# Problemdaten definieren
# ===================================================================

print("📊 FALLBEISPIEL: Automobilzulieferer GmbH")
print("=" * 60)

# Produktdaten
produkte = {
    'Bremssattel': {
        'D': 12000,    # Jahresbedarf (Stück)
        'K': 150,      # Bestellkosten (€)
        'h': 8.50      # Lagerkosten (€/Stück/Jahr)
    },
    'Stoßdämpfer': {
        'D': 8000,
        'K': 200,
        'h': 12.00
    },
    'Kupplungsscheibe': {
        'D': 15000,
        'K': 120,
        'h': 6.80
    }
}

# Daten als DataFrame anzeigen
df_produkte = pd.DataFrame(produkte).T
df_produkte.columns = ['Jahresbedarf (D)', 'Bestellkosten (K)', 'Lagerkosten (h)']
df_produkte.index.name = 'Produkt'

print("\n📋 PRODUKTDATEN:")
print("-" * 60)
print(df_produkte.to_string())

print("\n💡 ERKLÄRUNG DER KOSTENKOMPONENTEN:")
print("-" * 60)
print("   Bestellkosten (K): Fixkosten pro Bestellung")
print("      → Transport, Verwaltung, Wareneingang, Qualitätsprüfung")
print("\n   Lagerkosten (h): Kosten pro Stück und Jahr")
print("      → Kapitalbindung, Lagermiete, Versicherung, Schwund")

In [ ]:
# ===================================================================
# Optimale Losgrößen berechnen
# ===================================================================

print("\n🎯 OPTIMIERUNGSERGEBNISSE")
print("=" * 70)

ergebnisse = {}

for produkt, daten in produkte.items():
    erg = eoq_optimal(daten['D'], daten['K'], daten['h'])
    ergebnisse[produkt] = erg
    
    print(f"\n📦 {produkt}:")
    print("-" * 40)
    print(f"   Optimale Losgröße (Q*):     {erg['Q_optimal']:>10.0f} Stück")
    print(f"   Bestellkosten:              {erg['Bestellkosten']:>10.2f} €/Jahr")
    print(f"   Lagerkosten:                {erg['Lagerkosten']:>10.2f} €/Jahr")
    print(f"   Minimale Gesamtkosten:      {erg['Gesamtkosten']:>10.2f} €/Jahr")
    print(f"   Bestellhäufigkeit:          {erg['Bestellhaeufigkeit']:>10.1f} mal/Jahr")
    print(f"   Bestellzyklus:              {erg['Bestellzyklus_Tage']:>10.0f} Tage")
    print(f"   Durchschn. Lagerbestand:    {erg['Durchschn_Bestand']:>10.0f} Stück")

# Zusammenfassende Tabelle
print("\n" + "=" * 70)
print("📊 ZUSAMMENFASSUNG:")
print("-" * 70)

zusammenfassung = []
for produkt, erg in ergebnisse.items():
    zusammenfassung.append({
        'Produkt': produkt,
        'Q* (Stück)': f"{erg['Q_optimal']:.0f}",
        'Kosten (€/Jahr)': f"{erg['Gesamtkosten']:.2f}",
        'Bestellungen/Jahr': f"{erg['Bestellhaeufigkeit']:.1f}",
        'Zyklus (Tage)': f"{erg['Bestellzyklus_Tage']:.0f}"
    })

df_zusammenfassung = pd.DataFrame(zusammenfassung)
print(df_zusammenfassung.to_string(index=False))

# Verifikation: Bestellkosten = Lagerkosten
print("\n✅ VERIFIKATION (Bestellkosten ≈ Lagerkosten bei Q*):")
for produkt, erg in ergebnisse.items():
    diff = abs(erg['Bestellkosten'] - erg['Lagerkosten'])
    print(f"   {produkt}: Differenz = {diff:.2f} € (sollte ≈ 0 sein)")

---

## 4. Visualisierung der Kostenfunktion

Die Visualisierung zeigt den **Trade-off** zwischen Bestell- und Lagerkosten sowie das **Kostenminimum** bei der optimalen Losgröße.

In [ ]:
# ===================================================================
# Visualisierung der Kostenfunktion
# ===================================================================

def plot_kostenfunktion(produkt_name, D, K, h):
    """
    Visualisiert die Kostenfunktion und das Optimum für ein Produkt.
    
    Parameter:
    ----------
    produkt_name : str
        Name des Produkts
    D, K, h : float
        EOQ-Parameter
    """
    
    # Optimum berechnen
    erg = eoq_optimal(D, K, h)
    Q_opt = erg['Q_optimal']
    kosten_opt = erg['Gesamtkosten']
    
    # Q-Bereich für Plot (von 10% bis 300% des Optimums)
    Q_range = np.linspace(Q_opt * 0.1, Q_opt * 3, 500)
    
    # Kostenkomponenten berechnen
    bestellkosten = (D / Q_range) * K
    lagerkosten = (Q_range / 2) * h
    gesamtkosten = bestellkosten + lagerkosten
    
    # Plot erstellen
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Kostenlinien
    ax.plot(Q_range, bestellkosten, 'b--', linewidth=2, 
           label=f'Bestellkosten: (D/Q)·K')
    ax.plot(Q_range, lagerkosten, 'r--', linewidth=2, 
           label=f'Lagerkosten: (Q/2)·h')
    ax.plot(Q_range, gesamtkosten, 'g-', linewidth=3, 
           label='Gesamtkosten')
    
    # Optimum markieren
    ax.axvline(x=Q_opt, color='orange', linestyle=':', linewidth=2, alpha=0.8)
    ax.plot(Q_opt, kosten_opt, 'ko', markersize=12, zorder=5)
    ax.plot(Q_opt, kosten_opt, 'yo', markersize=8, zorder=6,
           label=f'Optimum: Q* = {Q_opt:.0f}, K* = {kosten_opt:.0f} €')
    
    # Annotation
    ax.annotate(f'Q* = {Q_opt:.0f} Stück\nK* = {kosten_opt:.0f} €/Jahr',
               xy=(Q_opt, kosten_opt),
               xytext=(Q_opt * 1.3, kosten_opt * 1.2),
               fontsize=11, fontweight='bold',
               arrowprops=dict(arrowstyle='->', color='black'),
               bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    # Schnittpunkt markieren (Bestellkosten = Lagerkosten)
    ax.plot(Q_opt, erg['Bestellkosten'], 'bs', markersize=8)
    ax.plot(Q_opt, erg['Lagerkosten'], 'rs', markersize=8)
    
    # Formatierung
    ax.set_xlabel('Losgröße Q (Stück)', fontsize=12)
    ax.set_ylabel('Kosten (€/Jahr)', fontsize=12)
    ax.set_title(f'EOQ-Kostenanalyse: {produkt_name}', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, Q_opt * 3)
    ax.set_ylim(0, kosten_opt * 2.5)
    
    plt.tight_layout()
    plt.show()
    
    # Zusätzliche Informationen
    print(f"\n💡 INTERPRETATION für {produkt_name}:")
    print(f"   • Bei Q < {Q_opt:.0f}: Zu häufige Bestellungen → hohe Bestellkosten")
    print(f"   • Bei Q > {Q_opt:.0f}: Zu große Bestände → hohe Lagerkosten")
    print(f"   • Bei Q* = {Q_opt:.0f}: Optimaler Kompromiss → minimale Gesamtkosten")

# Plot für Bremssattel
print("📈 VISUALISIERUNG DER KOSTENFUNKTION")
print("=" * 50)
plot_kostenfunktion('Bremssattel', 
                   produkte['Bremssattel']['D'],
                   produkte['Bremssattel']['K'],
                   produkte['Bremssattel']['h'])

In [ ]:
# ===================================================================
# Vergleich aller drei Produkte
# ===================================================================

def plot_vergleich_alle_produkte(produkte_dict, ergebnisse_dict):
    """
    Vergleicht die Kostenfunktionen aller Produkte.
    """
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    farben = ['#2E86AB', '#A23B72', '#F18F01']
    
    for idx, (produkt, daten) in enumerate(produkte_dict.items()):
        ax = axes[idx]
        erg = ergebnisse_dict[produkt]
        Q_opt = erg['Q_optimal']
        
        # Q-Bereich
        Q_range = np.linspace(Q_opt * 0.2, Q_opt * 2.5, 300)
        gesamtkosten = [(daten['D']/Q)*daten['K'] + (Q/2)*daten['h'] for Q in Q_range]
        
        # Plot
        ax.plot(Q_range, gesamtkosten, color=farben[idx], linewidth=2)
        ax.axvline(x=Q_opt, color='red', linestyle='--', alpha=0.7)
        ax.plot(Q_opt, erg['Gesamtkosten'], 'ro', markersize=10)
        
        ax.set_xlabel('Losgröße Q')
        ax.set_ylabel('Gesamtkosten (€/Jahr)')
        ax.set_title(f'{produkt}\nQ* = {Q_opt:.0f}, K* = {erg["Gesamtkosten"]:.0f} €')
        ax.grid(True, alpha=0.3)
    
    plt.suptitle('Vergleich der Kostenfunktionen', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

plot_vergleich_alle_produkte(produkte, ergebnisse)

---

## 5. Sensitivitätsanalyse

### Warum Sensitivitätsanalyse?

In der Praxis sind die Parameter **nicht exakt bekannt**:
- Der Bedarf $D$ kann schwanken
- Bestellkosten $K$ können sich ändern (z.B. neue Lieferanten)
- Lagerkosten $h$ sind oft Schätzungen

### Analytische Sensitivität

Aus der EOQ-Formel $Q^* = \sqrt{\frac{2DK}{h}}$ ergibt sich:

| Parameter-Änderung | Auswirkung auf Q* |
|--------------------|-------------------|
| D verdoppelt sich | Q* steigt um Faktor $\sqrt{2} \approx 1.41$ |
| K verdoppelt sich | Q* steigt um Faktor $\sqrt{2} \approx 1.41$ |
| h verdoppelt sich | Q* sinkt um Faktor $\sqrt{2} \approx 0.71$ |

**Wichtige Erkenntnis:** Die Kostenfunktion ist in der Nähe des Optimums **relativ flach** → kleine Abweichungen von Q* haben geringe Kostenauswirkungen!

In [ ]:
# ===================================================================
# Sensitivitätsanalyse
# ===================================================================

def sensitivitaetsanalyse(D, K, h, parameter='K', variation_prozent=30):
    """
    Führt eine Sensitivitätsanalyse für einen Parameter durch.
    
    Parameter:
    ----------
    D, K, h : float
        Basisparameter
    parameter : str
        Zu variierender Parameter ('D', 'K', oder 'h')
    variation_prozent : float
        Prozentuale Variation nach oben und unten
    """
    
    # Basisfall
    basis = eoq_optimal(D, K, h)
    Q_basis = basis['Q_optimal']
    kosten_basis = basis['Gesamtkosten']
    
    # Parametervariationen
    param_dict = {'D': D, 'K': K, 'h': h}
    param_namen = {'D': 'Jahresbedarf D (Stück)', 
                   'K': 'Bestellkosten K (€)', 
                   'h': 'Lagerkosten h (€/Stück/Jahr)'}
    
    basis_wert = param_dict[parameter]
    variationen = np.linspace(basis_wert * (1 - variation_prozent/100),
                             basis_wert * (1 + variation_prozent/100), 31)
    
    # Ergebnisse berechnen
    Q_werte = []
    kosten_werte = []
    
    for var_wert in variationen:
        if parameter == 'D':
            erg = eoq_optimal(var_wert, K, h)
        elif parameter == 'K':
            erg = eoq_optimal(D, var_wert, h)
        else:  # h
            erg = eoq_optimal(D, K, var_wert)
        
        Q_werte.append(erg['Q_optimal'])
        kosten_werte.append(erg['Gesamtkosten'])
    
    # Visualisierung
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Optimale Losgröße
    ax1 = axes[0]
    ax1.plot(variationen, Q_werte, 'b-', linewidth=2)
    ax1.axvline(x=basis_wert, color='r', linestyle='--', linewidth=2, alpha=0.7,
               label=f'Basis: {basis_wert}')
    ax1.axhline(y=Q_basis, color='g', linestyle=':', linewidth=2, alpha=0.7,
               label=f'Q* Basis: {Q_basis:.0f}')
    ax1.scatter([basis_wert], [Q_basis], color='red', s=100, zorder=5)
    ax1.set_xlabel(param_namen[parameter], fontsize=11)
    ax1.set_ylabel('Optimale Losgröße Q* (Stück)', fontsize=11)
    ax1.set_title('Einfluss auf optimale Losgröße', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Minimale Kosten
    ax2 = axes[1]
    ax2.plot(variationen, kosten_werte, 'g-', linewidth=2)
    ax2.axvline(x=basis_wert, color='r', linestyle='--', linewidth=2, alpha=0.7,
               label=f'Basis: {basis_wert}')
    ax2.axhline(y=kosten_basis, color='b', linestyle=':', linewidth=2, alpha=0.7,
               label=f'K* Basis: {kosten_basis:.0f} €')
    ax2.scatter([basis_wert], [kosten_basis], color='red', s=100, zorder=5)
    ax2.set_xlabel(param_namen[parameter], fontsize=11)
    ax2.set_ylabel('Minimale Gesamtkosten (€/Jahr)', fontsize=11)
    ax2.set_title('Einfluss auf minimale Kosten', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Elastizitäten berechnen
    delta_param = 0.01  # 1% Änderung
    if parameter == 'D':
        Q_plus = eoq_optimal(D * (1 + delta_param), K, h)['Q_optimal']
    elif parameter == 'K':
        Q_plus = eoq_optimal(D, K * (1 + delta_param), h)['Q_optimal']
    else:
        Q_plus = eoq_optimal(D, K, h * (1 + delta_param))['Q_optimal']
    
    elastizitaet = ((Q_plus - Q_basis) / Q_basis) / delta_param
    
    print(f"\n📊 SENSITIVITÄTSERGEBNISSE für Parameter {parameter}:")
    print(f"   Elastizität von Q* bezüglich {parameter}: {elastizitaet:.2f}")
    print(f"   → 1% Änderung in {parameter} führt zu ca. {elastizitaet:.2f}% Änderung in Q*")

# Sensitivitätsanalyse für Bremssattel
print("🔍 SENSITIVITÄTSANALYSE: Bremssattel")
print("=" * 50)

In [ ]:
# Variation der Bestellkosten K
print("\n▶ Variation der Bestellkosten K:")
sensitivitaetsanalyse(produkte['Bremssattel']['D'],
                     produkte['Bremssattel']['K'],
                     produkte['Bremssattel']['h'],
                     parameter='K')

In [ ]:
# Variation der Lagerkosten h
print("\n▶ Variation der Lagerkosten h:")
sensitivitaetsanalyse(produkte['Bremssattel']['D'],
                     produkte['Bremssattel']['K'],
                     produkte['Bremssattel']['h'],
                     parameter='h')

In [ ]:
# ===================================================================
# Robustheit der Lösung analysieren
# ===================================================================

def analysiere_robustheit(D, K, h, abweichung_prozent=20):
    """
    Analysiert die Kostenauswirkung bei Abweichung von Q*.
    """
    
    erg = eoq_optimal(D, K, h)
    Q_opt = erg['Q_optimal']
    K_opt = erg['Gesamtkosten']
    
    print("\n📊 ROBUSTHEITSANALYSE")
    print("=" * 60)
    print(f"Optimale Losgröße Q* = {Q_opt:.0f} Stück")
    print(f"Minimale Kosten K* = {K_opt:.2f} €/Jahr")
    print("\nKostenauswirkung bei Abweichung von Q*:")
    print("-" * 60)
    print(f"{'Abweichung':>12} {'Q':>10} {'Kosten':>15} {'Mehrkosten':>15} {'Mehrkosten %':>12}")
    print("-" * 60)
    
    abweichungen = [-50, -30, -20, -10, 0, 10, 20, 30, 50]
    
    for abw in abweichungen:
        Q_test = Q_opt * (1 + abw/100)
        kosten_test = eoq_kosten(Q_test, D, K, h)['Gesamtkosten']
        mehrkosten = kosten_test - K_opt
        mehrkosten_pct = (mehrkosten / K_opt) * 100
        
        marker = '◄── Optimum' if abw == 0 else ''
        print(f"{abw:>+10}% {Q_test:>10.0f} {kosten_test:>15.2f} € {mehrkosten:>+14.2f} € {mehrkosten_pct:>+11.2f}% {marker}")
    
    print("-" * 60)
    print("\n💡 ERKENNTNIS:")
    print("   Die Kostenfunktion ist in der Nähe des Optimums relativ flach!")
    print("   → Kleine Abweichungen von Q* verursachen nur geringe Mehrkosten.")
    print("   → Das EOQ-Modell ist robust gegenüber Schätzfehlern.")

analysiere_robustheit(produkte['Bremssattel']['D'],
                     produkte['Bremssattel']['K'],
                     produkte['Bremssattel']['h'])

---

## 6. Erweiterungen des EOQ-Modells

### 6.1 EOQ mit Mengenrabatten

In der Praxis bieten Lieferanten oft **Mengenrabatte** an:

| Bestellmenge | Stückpreis |
|--------------|------------|
| 0 - 499 | 25,00 € |
| 500 - 999 | 24,50 € (2% Rabatt) |
| 1.000 - 1.999 | 24,00 € (4% Rabatt) |
| ≥ 2.000 | 23,50 € (6% Rabatt) |

Die Gesamtkosten müssen dann um die **Warenkosten** erweitert werden:

$$K_{Gesamt} = \underbrace{\frac{D}{Q} \cdot K}_{\text{Bestellkosten}} + \underbrace{\frac{Q}{2} \cdot h}_{\text{Lagerkosten}} + \underbrace{D \cdot p}_{\text{Warenkosten}}$$

In [ ]:
# ===================================================================
# EOQ mit Mengenrabatten
# ===================================================================

def eoq_mit_mengenrabatt(D, K, h_basis, rabatt_stufen, zinssatz=0.10):
    """
    EOQ-Modell mit Mengenrabatten.
    
    Parameter:
    ----------
    D : float
        Jahresbedarf
    K : float
        Bestellkosten pro Bestellung
    h_basis : float
        Basis-Lagerhaltungskosten (ohne Kapitalbindung)
    rabatt_stufen : list of tuples
        Liste von (Mindestmenge, Stückpreis)
    zinssatz : float
        Zinssatz für Kapitalbindung (default: 10%)
        
    Returns:
    --------
    dict : Optimale Lösung
    """
    
    ergebnisse = []
    
    for i, (mindestmenge, preis) in enumerate(rabatt_stufen):
        # Effektive Lagerkosten = Basis + Kapitalbindung
        h_effektiv = h_basis + preis * zinssatz
        
        # Optimale Losgröße für diesen Preis
        Q_opt_theoretisch = np.sqrt((2 * D * K) / h_effektiv)
        
        # Bestimme den gültigen Q-Bereich für diese Rabattstufe
        min_q = mindestmenge
        max_q = rabatt_stufen[i+1][0] - 1 if i < len(rabatt_stufen) - 1 else float('inf')
        
        # Mögliche Q-Werte für diese Stufe
        kandidaten = []
        
        # Fall 1: Q* liegt im gültigen Bereich
        if min_q <= Q_opt_theoretisch <= max_q:
            kandidaten.append(Q_opt_theoretisch)
        
        # Fall 2: Mindestmenge dieser Stufe
        if min_q > 0:
            kandidaten.append(min_q)
        
        # Kosten für jeden Kandidaten berechnen
        for Q in kandidaten:
            bestellkosten = (D / Q) * K
            lagerkosten = (Q / 2) * h_effektiv
            warenkosten = D * preis
            gesamtkosten = bestellkosten + lagerkosten + warenkosten
            
            ergebnisse.append({
                'Rabattstufe': i + 1,
                'Mindestmenge': mindestmenge,
                'Stückpreis': preis,
                'Q': Q,
                'Q_theoretisch': Q_opt_theoretisch,
                'Bestellkosten': bestellkosten,
                'Lagerkosten': lagerkosten,
                'Warenkosten': warenkosten,
                'Gesamtkosten': gesamtkosten
            })
    
    # Beste Option finden
    df_ergebnisse = pd.DataFrame(ergebnisse)
    beste_idx = df_ergebnisse['Gesamtkosten'].idxmin()
    beste_option = df_ergebnisse.loc[beste_idx].to_dict()
    
    return beste_option, df_ergebnisse

# Beispiel Mengenrabatte für Bremssattel
print("💰 EOQ MIT MENGENRABATTEN: Bremssattel")
print("=" * 65)

rabatt_stufen = [
    (0, 25.00),      # Normalpreis
    (500, 24.50),    # 2% Rabatt ab 500 Stück
    (1000, 24.00),   # 4% Rabatt ab 1000 Stück
    (2000, 23.50)    # 6% Rabatt ab 2000 Stück
]

print("\n📋 RABATTSTAFFEL:")
print("-" * 40)
print(f"{'Mindestmenge':<15} {'Stückpreis':<15} {'Rabatt':<10}")
for menge, preis in rabatt_stufen:
    rabatt = (1 - preis/25) * 100
    print(f"{menge:<15} {preis:<15.2f} € {rabatt:<10.0f}%")

beste_option, df_analyse = eoq_mit_mengenrabatt(
    D=produkte['Bremssattel']['D'],
    K=produkte['Bremssattel']['K'],
    h_basis=5.0,  # Basis-Lagerkosten ohne Kapitalbindung
    rabatt_stufen=rabatt_stufen
)

print("\n📊 ANALYSE ALLER OPTIONEN:")
print("-" * 90)
display_cols = ['Rabattstufe', 'Mindestmenge', 'Stückpreis', 'Q', 'Warenkosten', 'Bestellkosten', 'Lagerkosten', 'Gesamtkosten']
df_display = df_analyse[display_cols].copy()
df_display['Q'] = df_display['Q'].apply(lambda x: f"{x:.0f}")
for col in ['Warenkosten', 'Bestellkosten', 'Lagerkosten', 'Gesamtkosten']:
    df_display[col] = df_display[col].apply(lambda x: f"{x:,.0f} €")
print(df_display.to_string(index=False))

print("\n🏆 OPTIMALE LÖSUNG:")
print("-" * 50)
print(f"   Rabattstufe:     {beste_option['Rabattstufe']:.0f} (Stückpreis: {beste_option['Stückpreis']:.2f} €)")
print(f"   Losgröße Q:      {beste_option['Q']:.0f} Stück")
print(f"   Gesamtkosten:    {beste_option['Gesamtkosten']:,.2f} €/Jahr")

# Vergleich mit Standard-EOQ
erg_standard = eoq_optimal(produkte['Bremssattel']['D'], 
                          produkte['Bremssattel']['K'], 
                          produkte['Bremssattel']['h'])
print(f"\n💡 Vergleich mit Standard-EOQ (ohne Rabatte):")
print(f"   Standard Q*:     {erg_standard['Q_optimal']:.0f} Stück")
print(f"   Mit Rabatt:      {beste_option['Q']:.0f} Stück → größere Lose lohnen sich!")

### 6.2 EOQ mit Sicherheitsbestand

In der Praxis ist die Nachfrage **unsicher**. Ein **Sicherheitsbestand** schützt vor Fehlmengen.

$$\text{Sicherheitsbestand} = z \cdot \sigma_L$$

wobei:
- $z$ = Sicherheitsfaktor (abhängig vom gewünschten Servicegrad)
- $\sigma_L$ = Standardabweichung der Nachfrage während der Lieferzeit

| Servicegrad | z-Faktor |
|-------------|----------|
| 90% | 1,28 |
| 95% | 1,65 |
| 99% | 2,33 |

In [ ]:
# ===================================================================
# EOQ mit Sicherheitsbestand
# ===================================================================

def eoq_mit_sicherheitsbestand(D, K, h, sigma_tag, lieferzeit_tage, servicegrad=0.95):
    """
    EOQ-Modell mit Sicherheitsbestand.
    
    Parameter:
    ----------
    D : float
        Jahresbedarf
    K : float
        Bestellkosten
    h : float
        Lagerhaltungskosten
    sigma_tag : float
        Standardabweichung der täglichen Nachfrage
    lieferzeit_tage : float
        Lieferzeit in Tagen
    servicegrad : float
        Gewünschter Servicegrad (default: 95%)
    """
    
    # z-Faktor aus Servicegrad
    from scipy.stats import norm
    z = norm.ppf(servicegrad)
    
    # Standardabweichung während Lieferzeit
    sigma_L = sigma_tag * np.sqrt(lieferzeit_tage)
    
    # Sicherheitsbestand
    sicherheitsbestand = z * sigma_L
    
    # Standard-EOQ
    Q_opt = np.sqrt((2 * D * K) / h)
    
    # Bestellpunkt (Reorder Point)
    tagesbedarf = D / 365
    bestellpunkt = tagesbedarf * lieferzeit_tage + sicherheitsbestand
    
    # Kosten des Sicherheitsbestands
    kosten_sicherheitsbestand = sicherheitsbestand * h
    
    # Gesamtkosten
    kosten_basis = eoq_kosten(Q_opt, D, K, h)['Gesamtkosten']
    gesamtkosten = kosten_basis + kosten_sicherheitsbestand
    
    return {
        'Q_optimal': Q_opt,
        'Sicherheitsbestand': sicherheitsbestand,
        'Bestellpunkt': bestellpunkt,
        'z_Faktor': z,
        'Kosten_Sicherheitsbestand': kosten_sicherheitsbestand,
        'Gesamtkosten_mit_SB': gesamtkosten,
        'Servicegrad': servicegrad
    }

# Beispiel für Bremssattel
print("🛡️ EOQ MIT SICHERHEITSBESTAND: Bremssattel")
print("=" * 65)

# Annahmen
sigma_tag = 15  # Standardabweichung der täglichen Nachfrage
lieferzeit = 5  # Lieferzeit in Tagen

print(f"\n📋 PARAMETER:")
print(f"   Jahresbedarf D:          {produkte['Bremssattel']['D']} Stück")
print(f"   Tagesbedarf:             {produkte['Bremssattel']['D']/365:.1f} Stück")
print(f"   Std.abw. Tagesbedarf:    {sigma_tag} Stück")
print(f"   Lieferzeit:              {lieferzeit} Tage")

# Verschiedene Servicegrade vergleichen
print("\n📊 VERGLEICH VERSCHIEDENER SERVICEGRADE:")
print("-" * 70)
print(f"{'Servicegrad':>12} {'z-Faktor':>10} {'Sich.bestand':>14} {'Bestellpunkt':>14} {'Mehrkosten':>12}")
print("-" * 70)

for sg in [0.90, 0.95, 0.99]:
    erg = eoq_mit_sicherheitsbestand(
        produkte['Bremssattel']['D'],
        produkte['Bremssattel']['K'],
        produkte['Bremssattel']['h'],
        sigma_tag, lieferzeit, sg
    )
    print(f"{sg*100:>11.0f}% {erg['z_Faktor']:>10.2f} {erg['Sicherheitsbestand']:>14.0f} {erg['Bestellpunkt']:>14.0f} {erg['Kosten_Sicherheitsbestand']:>11.0f} €")

print("-" * 70)

print("\n💡 INTERPRETATION:")
print("   • Höherer Servicegrad erfordert höheren Sicherheitsbestand")
print("   • Die Kosten steigen überproportional mit dem Servicegrad")
print("   • Trade-off: Lieferbereitschaft vs. Kapitalbindung")

---

## 7. Aufgaben für Studierende

Bearbeiten Sie die folgenden Aufgaben, um Ihr Verständnis der Losgrößenoptimierung zu vertiefen.

### ✏️ Aufgabe 1: Neues Produkt analysieren

Ein neues Produkt **"Radlager"** soll in das Sortiment aufgenommen werden:

| Parameter | Wert |
|-----------|------|
| Jahresbedarf (D) | 18.000 Stück |
| Bestellkosten (K) | 180 € pro Bestellung |
| Lagerkosten (h) | 9,50 €/Stück/Jahr |

**Aufgaben:**
1. Berechnen Sie die optimale Losgröße Q*
2. Berechnen Sie die minimalen Gesamtkosten
3. Wie oft muss pro Jahr bestellt werden?
4. Wie lange reicht eine Lieferung (in Tagen)?

In [ ]:
# Ihre Lösung für Aufgabe 1:
# ===========================

# TODO: Definieren Sie die Parameter
# TODO: Berechnen Sie die optimale Losgröße
# TODO: Geben Sie alle Kennzahlen aus



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 1 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 1: Neues Produkt 'Radlager'")
print("=" * 60)

# Parameter definieren
D_radlager = 18000  # Jahresbedarf
K_radlager = 180    # Bestellkosten
h_radlager = 9.50   # Lagerkosten

# Manuelle Berechnung zur Veranschaulichung
print("\n1️⃣ Manuelle Berechnung:")
print("-" * 40)
print(f"   Q* = √(2·D·K / h)")
print(f"   Q* = √(2 · {D_radlager} · {K_radlager} / {h_radlager})")
print(f"   Q* = √({2 * D_radlager * K_radlager} / {h_radlager})")
print(f"   Q* = √{(2 * D_radlager * K_radlager) / h_radlager:.2f}")

Q_manuell = np.sqrt((2 * D_radlager * K_radlager) / h_radlager)
print(f"   Q* = {Q_manuell:.2f} Stück")

# Mit Funktion berechnen
print("\n2️⃣ Berechnung mit Funktion:")
print("-" * 40)

erg_radlager = eoq_optimal(D_radlager, K_radlager, h_radlager)

print(f"   Optimale Losgröße Q*:       {erg_radlager['Q_optimal']:>10.0f} Stück")
print(f"   Minimale Gesamtkosten:      {erg_radlager['Gesamtkosten']:>10.2f} €/Jahr")
print(f"   Bestellhäufigkeit:          {erg_radlager['Bestellhaeufigkeit']:>10.1f} mal/Jahr")
print(f"   Bestellzyklus (Reichweite): {erg_radlager['Bestellzyklus_Tage']:>10.0f} Tage")

# Verifikation
print("\n3️⃣ Verifikation (Bestellkosten = Lagerkosten):")
print("-" * 40)
print(f"   Bestellkosten: {erg_radlager['Bestellkosten']:.2f} €/Jahr")
print(f"   Lagerkosten:   {erg_radlager['Lagerkosten']:.2f} €/Jahr")
print(f"   Differenz:     {abs(erg_radlager['Bestellkosten'] - erg_radlager['Lagerkosten']):.2f} € ✓")

### ✏️ Aufgabe 2: Was-wäre-wenn Analyse

Analysieren Sie für das **Bremssattel**-Beispiel die Auswirkungen folgender Szenarien:

| Szenario | Änderung |
|----------|----------|
| a) | Bestellkosten steigen um 50% |
| b) | Lagerkosten sinken um 30% |
| c) | Jahresbedarf steigt um 25% |

**Fragen:**
1. Wie ändert sich die optimale Losgröße?
2. Wie ändern sich die minimalen Kosten?
3. Welches Szenario hat den größten Einfluss?

In [ ]:
# Ihre Lösung für Aufgabe 2:
# ===========================

# TODO: Berechnen Sie den Basisfall
# TODO: Berechnen Sie alle drei Szenarien
# TODO: Vergleichen Sie die Ergebnisse



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 2 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 2: Was-wäre-wenn Analyse")
print("=" * 70)

# Basisdaten
D_basis = produkte['Bremssattel']['D']
K_basis = produkte['Bremssattel']['K']
h_basis = produkte['Bremssattel']['h']

# Basisfall
basis = eoq_optimal(D_basis, K_basis, h_basis)

# Szenarien
szenarien = {
    'Basis': {'D': D_basis, 'K': K_basis, 'h': h_basis},
    'a) K +50%': {'D': D_basis, 'K': K_basis * 1.5, 'h': h_basis},
    'b) h -30%': {'D': D_basis, 'K': K_basis, 'h': h_basis * 0.7},
    'c) D +25%': {'D': D_basis * 1.25, 'K': K_basis, 'h': h_basis}
}

print("\n📊 VERGLEICH DER SZENARIEN:")
print("-" * 75)
print(f"{'Szenario':<15} {'D':>10} {'K':>10} {'h':>10} {'Q*':>10} {'K*':>12} {'ΔQ*':>8} {'ΔK*':>8}")
print("-" * 75)

for name, params in szenarien.items():
    erg = eoq_optimal(params['D'], params['K'], params['h'])
    
    if name == 'Basis':
        delta_Q = '-'
        delta_K = '-'
    else:
        delta_Q = f"{(erg['Q_optimal']/basis['Q_optimal'] - 1)*100:+.1f}%"
        delta_K = f"{(erg['Gesamtkosten']/basis['Gesamtkosten'] - 1)*100:+.1f}%"
    
    print(f"{name:<15} {params['D']:>10} {params['K']:>10.0f} {params['h']:>10.2f} {erg['Q_optimal']:>10.0f} {erg['Gesamtkosten']:>12.2f} {delta_Q:>8} {delta_K:>8}")

print("-" * 75)

print("\n💡 ERKENNTNISSE:")
print("   a) K +50%: Q* steigt um ~22%, Kosten steigen um ~22%")
print("   b) h -30%: Q* steigt um ~20%, Kosten sinken um ~16%")
print("   c) D +25%: Q* steigt um ~12%, Kosten steigen um ~12%")
print("\n   → Szenario a) (Bestellkosten +50%) hat den größten negativen Kosteneffekt")
print("   → Die Losgröße reagiert auf alle Parameter mit Wurzel-Funktion (gedämpft)")

### ✏️ Aufgabe 3: Mengenrabatt-Entscheidung

Für den **Stoßdämpfer** bietet ein Lieferant folgende Rabattstaffel:

| Bestellmenge | Stückpreis |
|--------------|------------|
| 0 - 299 | 45,00 € |
| 300 - 599 | 43,50 € |
| ≥ 600 | 42,00 € |

**Aufgaben:**
1. Berechnen Sie die optimale Bestellmenge unter Berücksichtigung der Rabatte
2. Lohnt sich der Mengenrabatt?
3. Wie hoch ist die jährliche Ersparnis im Vergleich zum Normalpreis?

In [ ]:
# Ihre Lösung für Aufgabe 3:
# ===========================

# TODO: Definieren Sie die Rabattstufen
# TODO: Berechnen Sie die optimale Lösung
# TODO: Vergleichen Sie mit der Lösung ohne Rabatt



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 3 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 3: Mengenrabatt-Entscheidung Stoßdämpfer")
print("=" * 70)

# Rabattstufen für Stoßdämpfer
rabatt_stoss = [
    (0, 45.00),
    (300, 43.50),
    (600, 42.00)
]

print("\n📋 RABATTSTAFFEL:")
print("-" * 40)
for menge, preis in rabatt_stoss:
    rabatt = (1 - preis/45) * 100
    print(f"   ab {menge:>4} Stück: {preis:>6.2f} € ({rabatt:>4.1f}% Rabatt)")

# Analyse mit Rabatt
beste, df_ana = eoq_mit_mengenrabatt(
    D=produkte['Stoßdämpfer']['D'],
    K=produkte['Stoßdämpfer']['K'],
    h_basis=5.0,
    rabatt_stufen=rabatt_stoss
)

print("\n📊 ANALYSE:")
print("-" * 80)
print(df_ana[['Rabattstufe', 'Mindestmenge', 'Stückpreis', 'Q', 'Gesamtkosten']].to_string(index=False))

# Vergleich mit Normalpreis (ohne Rabatt)
kosten_normalpreis = beste['Gesamtkosten'] if beste['Stückpreis'] == 45.00 else None

# Berechne Kosten bei Normalpreis
D_stoss = produkte['Stoßdämpfer']['D']
h_normal = 5.0 + 45.00 * 0.10  # Basis + Kapitalbindung
Q_normal = np.sqrt((2 * D_stoss * produkte['Stoßdämpfer']['K']) / h_normal)
kosten_normal = (D_stoss/Q_normal) * produkte['Stoßdämpfer']['K'] + (Q_normal/2) * h_normal + D_stoss * 45.00

ersparnis = kosten_normal - beste['Gesamtkosten']

print("\n🏆 ERGEBNIS:")
print("-" * 50)
print(f"   Optimale Bestellmenge: {beste['Q']:.0f} Stück")
print(f"   Optimaler Stückpreis:  {beste['Stückpreis']:.2f} € (Rabattstufe {beste['Rabattstufe']:.0f})")
print(f"\n   Kosten ohne Rabatt:    {kosten_normal:,.2f} €/Jahr")
print(f"   Kosten mit Rabatt:     {beste['Gesamtkosten']:,.2f} €/Jahr")
print(f"   ─────────────────────────────────────────")
print(f"   Jährliche Ersparnis:   {ersparnis:,.2f} € ({ersparnis/kosten_normal*100:.1f}%)")

print("\n💡 FAZIT:")
print(f"   Der Mengenrabatt lohnt sich! Durch Bestellung von {beste['Q']:.0f} Stück")
print(f"   statt {Q_normal:.0f} Stück spart das Unternehmen {ersparnis:,.0f} € pro Jahr.")

---

## 8. Zusammenfassung und Ausblick

### Was haben wir gelernt?

| Thema | Kernaussage |
|-------|-------------|
| **EOQ-Formel** | $Q^* = \sqrt{\frac{2DK}{h}}$ minimiert die Summe aus Bestell- und Lagerkosten |
| **Trade-off** | Große Lose → weniger Bestellungen, aber mehr Kapitalbindung |
| **Optimum** | Bei Q* sind Bestellkosten = Lagerkosten |
| **Robustheit** | Die Kostenfunktion ist flach → kleine Abweichungen sind unkritisch |
| **Sensitivität** | Q* reagiert mit Wurzel-Funktion auf Parameteränderungen |
| **Erweiterungen** | Mengenrabatte und Sicherheitsbestände erfordern erweiterte Modelle |

### Grenzen des EOQ-Modells

- ❌ Konstante Nachfrage (unrealistisch bei Saisonalität)
- ❌ Keine Kapazitätsbeschränkungen
- ❌ Keine Koordination mehrerer Produkte
- ❌ Keine dynamische Anpassung

### Weiterführende Themen

- **EPQ (Economic Production Quantity)**: Für Eigenproduktion statt Bestellung
- **Dynamische Losgrößenmodelle**: Wagner-Whitin, Silver-Meal bei schwankender Nachfrage
- **Mehrprodukt-Optimierung**: Koordinierte Bestellpolitik
- **Stochastische Modelle**: Bei unsicherer Nachfrage

In [ ]:
# ===================================================================
# Abschluss und Checkliste
# ===================================================================

print("\n" + "="*70)
print("📋 CHECKLISTE - Losgrößenoptimierung")
print("="*70)

checkliste = [
    ("EOQ-Formel verstehen und anwenden", 
     "Q* = √(2DK/h), K* = √(2DKh)"),
    ("Kostenkomponenten kennen", 
     "Bestellkosten (D/Q)·K, Lagerkosten (Q/2)·h"),
    ("Optimum charakterisieren", 
     "Bei Q* gilt: Bestellkosten = Lagerkosten"),
    ("Sensitivitätsanalyse durchführen", 
     "Einfluss von D, K, h auf Q* und K* quantifizieren"),
    ("Robustheit bewerten", 
     "Kostenfunktion ist flach → EOQ ist robust"),
    ("Erweiterungen kennen", 
     "Mengenrabatte, Sicherheitsbestand, dynamische Modelle"),
    ("Praxisbezug herstellen", 
     "Annahmen prüfen, Modellgrenzen kennen")
]

for i, (punkt, beschreibung) in enumerate(checkliste, 1):
    print(f"\n☐ {i}. {punkt}")
    print(f"   → {beschreibung}")

print("\n" + "="*70)
print("🎓 Viel Erfolg bei der Anwendung in der Praxis!")
print("   Nächstes Thema: Lineare Optimierung")
print("="*70)